<a href="https://colab.research.google.com/github/AsifRifat/Cricket-Outcome-Predictor/blob/main/CricketOutcomePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>